## UAS Desain dan Analisis Algoritma

In [15]:
#inisialisasi num_satpam (5 poin)
#contoh nidn: 193510672
#jumlah 3 digit terakhir ialah 1+0+1 = 2
num_satpam = int(input("Silakan hitung 3 digit terakhir dari nidn anda dan tuliskan hasilnya: "))
if num_satpam < 9:
    num_satpam = num_satpam + 8

Silakan hitung 3 digit terakhir dari nidn anda dan tuliskan hasilnya: 2


In [16]:
#print soal (5 poin)
print(f"Universitas Islam Riau memiliki {num_satpam} satpam yang bekerja beberapa shift. UIR perlu menjadwalkan satpam yang cukup untuk setiap shift harian. Biasanya, jadwal akan memiliki kendala, yaitu:\n1. Setiap hari dibagi menjadi 9 shift untuk menjaga di 9 fakultas berbeda.\n2. Setiap hari, setiap shift ditugaskan untuk satu satpam, dan tidak ada satpam yang bekerja lebih dari satu shift.\n3. Setiap satpam ditugaskan untuk setidaknya dua shift selama periode 7 hari.\nUIR kesulitan untuk menyusun jadwal yang memenuhi semua kendala dan meminta bantuan Anda untuk menyusun jadwal yang optimal.")

Universitas Islam Riau memiliki 10 satpam yang bekerja beberapa shift. UIR perlu menjadwalkan satpam yang cukup untuk setiap shift harian. Biasanya, jadwal akan memiliki kendala, yaitu:
1. Setiap hari dibagi menjadi 9 shift untuk menjaga di 9 fakultas berbeda.
2. Setiap hari, setiap shift ditugaskan untuk satu satpam, dan tidak ada satpam yang bekerja lebih dari satu shift.
3. Setiap satpam ditugaskan untuk setidaknya dua shift selama periode 7 hari.
UIR kesulitan untuk menyusun jadwal yang memenuhi semua kendala dan meminta bantuan Anda untuk menyusun jadwal yang optimal.


In [17]:
#Import model (2 poin)
from ortools.sat.python import cp_model

In [18]:
# Data (20 poin)
num_shifts = 2
num_days = 7
all_satpam = range(num_satpam)
all_shifts = range(num_shifts)
all_days = range(num_days)

In [19]:
# Creates the model. (2 poin)
model = cp_model.CpModel()

In [20]:
# Creates shift variables. (5 poin)
# shifts[(n, d, s)]: satpam 'n' works shift 's' on day 'd'.
shifts = {}
for n in all_satpam:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d,
                    s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

In [21]:
# CONSTRAINT 1 (5 poin)
# Each shift is assigned to exactly one satpam in the schedule period.
for d in all_days:
    for s in all_shifts:
        model.AddExactlyOne(shifts[(n, d, s)] for n in all_satpam)

In [22]:
# CONSTRAINT 2 (5 poin)
# Each satpam works at most one shift per day.
for n in all_satpam:
    for d in all_days:
        model.AddAtMostOne(shifts[(n, d, s)] for s in all_shifts)

In [23]:
# Try to distribute the shifts evenly, so that each satpam works (10 poin)
# min_shifts_per_satpam shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of satpam, some satpam will
# be assigned one more shift.
min_shifts_per_satpam = (num_shifts * num_days) // num_satpam
if num_shifts * num_days % num_satpam == 0:
    max_shifts_per_satpam = min_shifts_per_satpam
else:
    max_shifts_per_satpam = min_shifts_per_satpam + 1
for n in all_satpam:
    num_shifts_worked = []
    for d in all_days:
        for s in all_shifts:
            num_shifts_worked.append(shifts[(n, d, s)])
    model.Add(min_shifts_per_satpam <= sum(num_shifts_worked))
    model.Add(sum(num_shifts_worked) <= max_shifts_per_satpam)

In [24]:
# Creates the solver and solve. (2 poin)
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0


In [25]:
# Enumerate all solutions. (27 poin)
solver.parameters.enumerate_all_solutions = True

In [26]:
# Display the first five solutions. (10 poin)
class SatpamPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_satpam, num_days, num_shifts, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_satpam = num_satpam
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print('Solution %i' % self._solution_count)
        for d in range(self._num_days):
            print('Day %i' % d)
            for n in range(self._num_satpam):
                is_working = False
                for s in range(self._num_shifts):
                    if self.Value(self._shifts[(n, d, s)]):
                        is_working = True
                        print('  Satpam %i bekerja pada shift %i' % (n, s))
                if not is_working:
                    print('  Satpam {} tidak bekerja'.format(n))
        if self._solution_count >= self._solution_limit:
            print('Stop search after %i solutions' % self._solution_limit)
            self.StopSearch()

    def solution_count(self):
        return self._solution_count
solution_limit = 5
solution_printer = SatpamPartialSolutionPrinter(shifts, num_satpam,
                                                num_days, num_shifts,
                                                solution_limit)

In [27]:
# Statistics (2 poin)
solver.Solve(model, solution_printer)

Solution 1
Day 0
  Satpam 0 tidak bekerja
  Satpam 1 tidak bekerja
  Satpam 2 tidak bekerja
  Satpam 3 tidak bekerja
  Satpam 4 tidak bekerja
  Satpam 5 tidak bekerja
  Satpam 6 tidak bekerja
  Satpam 7 tidak bekerja
  Satpam 8 bekerja pada shift 1
  Satpam 9 bekerja pada shift 0
Day 1
  Satpam 0 tidak bekerja
  Satpam 1 tidak bekerja
  Satpam 2 tidak bekerja
  Satpam 3 tidak bekerja
  Satpam 4 tidak bekerja
  Satpam 5 tidak bekerja
  Satpam 6 bekerja pada shift 1
  Satpam 7 bekerja pada shift 0
  Satpam 8 tidak bekerja
  Satpam 9 tidak bekerja
Day 2
  Satpam 0 tidak bekerja
  Satpam 1 tidak bekerja
  Satpam 2 tidak bekerja
  Satpam 3 tidak bekerja
  Satpam 4 bekerja pada shift 1
  Satpam 5 bekerja pada shift 0
  Satpam 6 tidak bekerja
  Satpam 7 tidak bekerja
  Satpam 8 tidak bekerja
  Satpam 9 tidak bekerja
Day 3
  Satpam 0 tidak bekerja
  Satpam 1 bekerja pada shift 1
  Satpam 2 tidak bekerja
  Satpam 3 bekerja pada shift 0
  Satpam 4 tidak bekerja
  Satpam 5 tidak bekerja
  Satpam 

2